#Machine Translation with Trax

Note by Denis Rothman: The original notebook was split into cells.

[Reference Code](https://colab.research.google.com/github/google/trax/blob/master/trax/intro.ipynb)


In [7]:
#@title Installing Trax
import os
import numpy as np

!pip install -q -U trax
import trax

트랜스 포머 모델 생성

In [8]:
#@title Creating
# Pre-trained model config in gs://trax-ml/models/translation/ende_wmt32k.gin
model = trax.models.Transformer(
    input_vocab_size=33300,
    d_model=512, d_ff=2048,
    n_heads=8, n_encoder_layers=6, n_decoder_layers=6,
    max_len=2048, mode='predict')


사전 훈련 가중치로 모델 초기화하기

In [9]:
#@title Initializing the model using pre-trained weights
model.init_from_file('gs://trax-ml/models/translation/ende_wmt32k.pkl.gz',
                     weights_only=True)

문장 토큰화

In [10]:
#@title Tokenizing a sentence
sentence = 'I am only a machine but I have machine intelligence.'

tokenized = list(trax.data.tokenize(iter([sentence]),  # Operates on streams.
                                    vocab_dir='gs://trax-ml/vocabs/',
                                    vocab_file='ende_32k.subword'))[0]


- 트랜스포머 디코딩

트랜스포머 문장을 영어로 인코딩하고, 독일어로 디코딩할 것.    
모델과 그 가중치가 그 능력 집합을 구성

In [11]:
#@title Decoding from the Transformer
tokenized = tokenized[None, :]  # Add batch dimension.
tokenized_translation = trax.supervised.decoding.autoregressive_sample(
    model, tokenized, temperature=0.0)  # Higher temperature: more diverse results.


- 역 토큰화 및 번역 표시

In [12]:
#@title De-tokenizing and Displaying the Translation
tokenized_translation = tokenized_translation[0][:-1]  # Remove batch and EOS.
translation = trax.data.detokenize(tokenized_translation,
                                   vocab_dir='gs://trax-ml/vocabs/',
                                   vocab_file='ende_32k.subword')
print("The sentence:",sentence)
print("The translation:",translation)

The sentence: I am only a machine but I have machine intelligence.
The translation: Ich bin nur eine Maschine, aber ich habe Maschinenübersicht.
